In [2]:
import argparse
import logging

import tensorflow as tf

from dataset import get_datasets
from logistic import softmax, cross_entropy, accuracy

In [5]:
def sgd(params, grads, lr, bs):
    """
    stochastic gradient descent implementation
    args:
    - params [list[tensor]]: model params
    - grad [list[tensor]]: param gradient
    - lr [float]: learning rate
    - bs [int]: batch_size
    """
    for param, grad in zip(params, grads):
        param.assign_sub(lr * grad / bs)


def training_loop(lr):
    """
    training loop
    args:
    - lr [float]: learning rate
    returns:
    - mean_acc [tensor]: training accuracy
    - mean_loss [tensor]: training loss
    """
    accuracies = []
    losses = []
    for X, Y in train_dataset:
        with tf.GradientTape() as tape:
            # forward pass
            X = X / 255.0
            y_hat = model(X)
            # calculate loss
            one_hot = tf.one_hot(Y, 43)
            loss = cross_entropy(y_hat, one_hot)
            losses.append(tf.math.reduce_mean(loss))

            grads = tape.gradient(loss, [W, b])
            sgd([W, b], grads, lr, X.shape[0]) 

            acc = accuracy(y_hat, Y)
            accuracies.append(acc)
    mean_acc = tf.math.reduce_mean(tf.concat(accuracies, axis=0))
    mean_loss = tf.math.reduce_mean(losses)
    return mean_loss, mean_acc

def model(X):
    """
    logistic regression model
    """
    flatten_X = tf.reshape(X, (-1, W.shape[0]))
    return softmax(tf.matmul(flatten_X, W) + b)

    
def validation_loop():
    """
    loop through the validation dataset
    """
    accuracies = []
    for X, Y in val_dataset:
        X = X / 255.0
        y_hat = model(X)
        acc = accuracy(y_hat, Y)
        accuracies.append(acc)
    mean_acc = tf.math.reduce_mean(tf.concat(accuracies, axis=0))
    return mean_acc


def get_module_logger(mod_name):
    logger = logging.getLogger(mod_name)
    handler = logging.StreamHandler()
    formatter = logging.Formatter('%(asctime)s %(levelname)-8s %(message)s')
    handler.setFormatter(formatter)
    logger.addHandler(handler)
    logger.setLevel(logging.DEBUG)
    return logger


# if __name__  == '__main__':
#     logger = get_module_logger(__name__)
#     parser = argparse.ArgumentParser(description='Download and process tf files')
#     parser.add_argument('--imdir', required=True, type=str,
#                         help='data directory')
#     parser.add_argument('--epochs', default=10, type=int,
#                         help='Number of epochs')
#     args = parser.parse_args()    

#     logger.info(f'Training for {args.epochs} epochs using {args.imdir} data')
#     # get the datasets
#     train_dataset, val_dataset = get_datasets(args.imdir)

#     # set the variables
#     num_inputs = 1024*3
#     num_outputs = 43
#     W = tf.Variable(tf.random.normal(shape=(num_inputs, num_outputs),
#                                     mean=0, stddev=0.01))
#     b = tf.Variable(tf.zeros(num_outputs))
    
#     lr = 0.1
#     # training! 
#     for epoch in range(args.epochs):
#         logger.info(f'Epoch {epoch}')
#         loss, acc = training_loop(lr)
#         logger.info(f'Mean training loss: {loss:1f}, mean training accuracy {acc:1f}')
#         val_acc = validation_loop()
#         logger.info(f'Mean validation accuracy {val_acc:1f}')

In [20]:
epochs=5
imdir='./GTSRB'

In [9]:
logger = get_module_logger(__name__)
logger.info(f'Training for {epochs} epochs using {imdir} data')

2025-03-27 21:57:07,845 INFO     Training for 1 epochs using ./GTSRB data


In [11]:
train_dataset, val_dataset = get_datasets(imdir)

Found 4300 files belonging to 1 classes.
Using 3870 files for training.
Found 4300 files belonging to 1 classes.
Using 430 files for validation.


In [12]:
train_dataset

<BatchDataset element_spec=(TensorSpec(shape=(None, 32, 32, 3), dtype=tf.float32, name=None), TensorSpec(shape=(None,), dtype=tf.int32, name=None))>

In [13]:
val_dataset

<BatchDataset element_spec=(TensorSpec(shape=(None, 32, 32, 3), dtype=tf.float32, name=None), TensorSpec(shape=(None,), dtype=tf.int32, name=None))>

In [22]:
# set the variables
num_inputs = 1024*3
num_outputs = 43

W = tf.Variable(tf.random.normal(shape=(num_inputs, num_outputs),mean=0, stddev=0.01))
b = tf.Variable(tf.zeros(num_outputs))

In [24]:
W.shape

TensorShape([3072, 43])

In [25]:
b.shape

TensorShape([43])

In [26]:
lr = 0.01
epochs=5

# training! 
for epoch in range(epochs):
    logger.info(f'Epoch {epoch}')
    loss, acc = training_loop(lr)
    logger.info(f'Mean training loss: {loss:1f}, mean training accuracy {acc:1f}')
    val_acc = validation_loop()
    logger.info(f'Mean validation accuracy {val_acc:1f}')

2025-03-27 22:04:47,540 INFO     Epoch 0
2025-03-27 22:04:49,930 INFO     Mean training loss: 0.306262, mean training accuracy 0.983607
2025-03-27 22:04:50,179 INFO     Mean validation accuracy 1.000000
2025-03-27 22:04:50,179 INFO     Epoch 1
2025-03-27 22:04:53,668 INFO     Mean training loss: 0.086758, mean training accuracy 1.000000
2025-03-27 22:04:54,045 INFO     Mean validation accuracy 1.000000
2025-03-27 22:04:54,045 INFO     Epoch 2
2025-03-27 22:04:57,932 INFO     Mean training loss: 0.060019, mean training accuracy 1.000000
2025-03-27 22:04:58,326 INFO     Mean validation accuracy 1.000000
2025-03-27 22:04:58,330 INFO     Epoch 3
2025-03-27 22:05:02,242 INFO     Mean training loss: 0.047434, mean training accuracy 1.000000
2025-03-27 22:05:02,635 INFO     Mean validation accuracy 1.000000
2025-03-27 22:05:02,643 INFO     Epoch 4
2025-03-27 22:05:06,531 INFO     Mean training loss: 0.039319, mean training accuracy 1.000000
2025-03-27 22:05:06,885 INFO     Mean validation acc